# Setup

In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv
import os
import nest_asyncio

nest_asyncio.apply()

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

embed_model = OpenAIEmbedding(model_name="text-embedding-ada-002", api_key=OPENAI_API_KEY)
llm = OpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o-mini", temperature=0.1)

In [6]:
from vectorstore import get_vectorstore

vectorstore = get_vectorstore()

In [7]:
from index import get_index

index = get_index(embed_model=embed_model)

In [14]:
import json

def store_docs(docs, filename):
    docs_to_store = []

    for node in docs:
        docs_to_store.append({
            "text": node.text,
            "metadata": node.metadata,
            "score": node.score
        })

    with open(f"./output/{filename}", "w", encoding="utf-8") as f:
        json.dump(docs_to_store, f, indent=4)

# Retrieving

In [8]:
query_gen_prompt = """You are an AI language model assistant specializing in query expansion. Your task is to generate {num_queries} diverse versions of the given user question. These variations will be used to retrieve relevant documents from a vector database, helping to overcome limitations of distance-based similarity search.

Original question: {query}

Instructions:
1. Create {num_queries} unique variations of the original question.
2. Ensure each variation maintains the core intent of the original question.
3. Use different phrasings, synonyms, or perspectives for each variation.
4. Consider potential context or implications not explicitly stated in the original question.
5. Avoid introducing new topics or drastically changing the meaning of the question.

Please provide your {num_queries} question variations, each on a new line:
"""

In [9]:
top_n = 50
num_queries = 5
question = "What is our duty in this world?"

## Existing Implementation

In [10]:
vector_retriever = index.as_retriever(similarity_top_k=top_n)

In [10]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_retriever],
    similarity_top_k=top_n,
    num_queries=num_queries,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt,
    llm=llm,
)

In [69]:
# it will genereate n - 1 queries since the original query is also included
nodes_with_scores = retriever.retrieve(question)

Generated queries:
1. What responsibilities do we have in this world?
2. How can we fulfill our obligations in this world?
3. What role are we meant to play in this world?
4. What is expected of us in this world?


In [70]:
from llm_score_docs import BatchProcessingConfig, batch_score_documents

config = BatchProcessingConfig()
scored_docs = batch_score_documents(nodes_with_scores, question, config)

Processing batch 1
Processing batch 2
Processing batch 3
Processing batch 4
Raw result: ```json
[
    {
        "reason": "Discusses duty through faith and personal improvement.",
        "score": 9
    },
    {
        "reason": "Highlights individual responsibility and spiritual duty to society.",
        "score": 8
    },
    {
        "reason": "Emphasizes gratitude and earning a living as worship.",
        "score": 6
    },
    {
        "reason": "Describes ethical behavior and social responsibility in Islam.",
        "score": 7
    }
]
```
[INFO] Batch scored 4 contexts
Got the scores [(9, 'Discusses duty through faith and personal improvement.'), (8, 'Highlights individual responsibility and spiritual duty to society.'), (6, 'Emphasizes gratitude and earning a living as worship.'), (7, 'Describes ethical behavior and social responsibility in Islam.')]
Relevance score: 9 - Discusses duty through faith and personal improvement.
Text: You should always remember that Allah has pr

In [71]:
relevance_threshold = 4

filtered_docs = [
    doc for doc in scored_docs
    if doc.score >= relevance_threshold
]

len(filtered_docs)

49

In [73]:
from IPython.display import display, Markdown
import pyperclip

# Create a string to store all markdown content
markdown_output = ""

for i, doc in enumerate(filtered_docs):
    # Add content to the string
    markdown_output += f"\n## Document {i+1}\n\n"
    markdown_output += f"{doc.doc.text}\n"

# Copy to clipboard
pyperclip.copy(markdown_output)
print("Markdown content copied to clipboard!")

Markdown content copied to clipboard!


In [38]:
store_docs([doc.doc for doc in filtered_docs], "baseline_docs.json")

## Cross Encoder

In [2]:
import os
from huggingface_hub import snapshot_download
from llama_index.core.postprocessor import SentenceTransformerRerank

# Download model explicitly (one-time operation)
model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
cache_dir = './models'

# Download model files
_ = snapshot_download(
    repo_id=model_name,
    cache_dir=cache_dir,
    local_dir=os.path.join(cache_dir, model_name)
)

# Create reranker (will use cached model)
sentence_transformer_rerank = SentenceTransformerRerank(
    model="./models/cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_n=50
)

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 4511.62it/s]


In [11]:
a = SentenceTransformerRerank(
    model="./models/cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_n=50
)

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank

sentence_transformer_rerank = SentenceTransformerRerank(
    model='cross-encoder/ms-marco-MiniLM-L-6-v2',
    top_n=top_n
)

In [12]:
cross_encoder_retriever = index.as_retriever(similarity_top_k=top_n, postprocessor=a) # This will retrieve and rerank the top_n documents

In [13]:
docs = cross_encoder_retriever.retrieve(question)

In [ ]:
# a = index.as_retriever(similarity_top_k=top_n, postprocessors=[sentence_transformer_rerank])
# b = a.retrieve(question)

In [14]:
len(docs)

50

In [19]:
docs[0].node

TextNode(id_='ce2f8b57-1222-450e-810f-e0ac68b36719', embedding=None, metadata={'title': 'World of Our Youth', 'author': 'Sayyid Muhammad Husayn Fadlullah - XKP', 'language': 'en', 'description': "Youth in the light of Qur'an, essential models, responsibilities, and the Shari'a view.\nby: Ayatullah al-Udhma al-Sayyid Muhammad Husayn Fadlullah \nTranslated by: Khaleel Mohammed \nPublished by: Organization for the Advancement of Islamic Knowledge and Humanitarian Services \nMontreal, Canada", 'type': 'book', 'embeddings': 'openaiembeddings', 'id': 'epub-9ec6d83f82b34ec6b6f28fcc72138fe0'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='623cb6a6-e2b3-4200-86d4-e352cd2f49bd', node_type='4', metadata={'title': 'W

In [36]:
store_docs(docs, "cross_encoder_docs.json")

## Cross Encoder with Query Fusion

In [74]:
from llama_index.core.retrievers import QueryFusionRetriever

cross_encoder_retriever_with_query_fusion = QueryFusionRetriever(
    [cross_encoder_retriever],
    similarity_top_k=top_n,
    num_queries=num_queries,  # set this to 1 to disable query generation
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt,
    llm=llm,
)

In [75]:
docs2 = cross_encoder_retriever_with_query_fusion.retrieve(question)
len(docs2)

Generated queries:
1. What responsibilities do we have in the world we live in?
2. How can we fulfill our obligations in this world?
3. What role are we meant to play in the grand scheme of things on Earth?
4. In what ways can we contribute positively to the world around us?


50

In [76]:
from IPython.display import display, Markdown
import pyperclip

# Create a string to store all markdown content
markdown_output = ""

for i, doc in enumerate(docs2):
    # Add content to the string
    markdown_output += f"\n## Document {i+1}\n\n"
    markdown_output += f"{doc.text}\n"

# Copy to clipboard
pyperclip.copy(markdown_output)
print("Markdown content copied to clipboard!")

Markdown content copied to clipboard!


In [77]:
docs2[0].text

'****\n\n**Responsibilities of Those Whose Call is to Islam**\n\nIslam directs everyone who calls to Islam-the preacher, the guide-thus: "You\nmust give of everything you have in order to guide another, to change that\nperson\'s line of thinking, and to correct his way. When you do this, you will\nhave fulfilled your obligations in this respect." In the words of God: " ** _O\nyou who believe! Save yourselves and your families from fire whose fuel is\nhuman beings and stones, over which are angels, stern and sever They do not\ndisobey God in that which they are ordered, but do as they are commanded_** _"\n(al-Tahrim,_ 66:6). In the interpretation of this verse, a question was put\nforward to one of the  _Imams._ It was asked, "How do we save them?" He said,\n"Commanding them and prohibiting them." It was said, "We command and we\nprohibit, but they do not obey us." He said, "If you have commanded and\nprohibited them, you have fulfilled your duties."\n\nThe Imam, in his answer with resp

In [4]:
docs2[0].metadata

NameError: name 'docs2' is not defined

In [45]:
store_docs(docs2, "cross_encoder_retriever_with_query_fusion.json")